# Chap02 - 텐서플로 설치에서 실행까지

## 2.1 텐서플로 설치

```bash
# TensorFlow CPU 
$ pip install tensorflow

# TensorFlow GPU
$ pip install tensorflow-gpu
```

## 2.2 Hello World

TensorFlow의 첫 번째 예제인 `Hello`와 `World!`라는 단어를 결합해 `Hello World!`라는 문구를 출력하는 프로그램이다. 

In [1]:
import tensorflow as tf

# TensorFlow 버전 확인
print(tf.__version__)

C:\Users\donaldaq\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.0


In [2]:
# hello_world.py

h = tf.constant("Hello")  # 상수
w = tf.constant(" World!")
hw = h + w

with tf.Session() as sess:
    prt = sess.run(hw)
    
print(prt)

b'Hello World!'


다음은 `str`타입으로 `phw = ph + pw`와 `tf.constant`로 출력한 `hw`( h + w )를 비교해보자.

In [5]:
strh = "Hello"
strw = " World!"
strhw = strh + strw

print('str: {}, type: {}'.format(strhw, type(strhw)))
print('tensorflow: {}, type: {}'.format(hw, type(hw)))

str: Hello World!, type: <class 'str'>
tensorflow: Tensor("add:0", shape=(), dtype=string), type: <class 'tensorflow.python.framework.ops.Tensor'>


텐서플로의 연산 그래프와 관련된 중요한 개념은, 

1. 먼저 어떠한 연산을 할지 정의해둔 후
2. 외부 메커니즘을 통해서 그 연산을 실행 시킨다.

위의 코드에서 `hw = h + w`에서 텐서플로 코드는 `h`와 `w`의 합인 `Hello World!`를 출력하는 것이 아닌 나중에 실행될 합 연산을 **연산 그래프(computation graph)** 추가된다

`Session` 객체는 텐서플로 연산 메커니즘에 대한 인터페이스 역할이고, `prt = sess.run(hw)`에서 `hw = h + w`연산을 수행한다. 따라서, `print(prt)`를 하면 `Hello World!`를 출력함

## 2.3 MNIST

위에서 간단한 텐서플로 예제를 살펴 보았다. 이번에는 필기체 숫자로 이루어진 'MNIST 데이터베이스'를 이용해 필기체 숫자 분류를 해보자. MNIST는 미국 인구조사국의 직원들이 쓴 숫자와 고등학생들이 쓴 숫자로 만든 미국 국립표준기술연구소(NIST)의 데이터베이스를 다시 섞어 만든 필기체 숫자 이미지 데이터베이스이다. 

MNIST 데이터는 딥러닝 예제에서 빠지지 않고 등장하는 데이터라고 할 수 있다. 

![](./images/mnist.png)

## 2.4 소프트맥스 회귀 - Softmax Regression

이번에 구현할 MNIST 분류 예제에서는 **소프트맥스 회귀(Softmax Regression)** 으로 구현함

$$
h(x^{(i)}) = \frac{e^{w_{y_j}^Tx^{(i)}}} {\sum_{j = 1}^k e^{w_j^Tx^{(i)}}}
$$

In [6]:
# import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 상수 및 하이퍼파라미터 설정
DATA_DIR = '../data'
NUM_STEPS = 1000
MINIBATCH_SIZE = 100

# MNIST 데이터 불러오기
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../data\t10k-images-idx3-ubyte.gz
Extracting ../data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
#########
# 회귀 모델 만들기 
#########
x = tf.placeholder(tf.float32, [None, 784]) # 28 * 28
W = tf.Variable(tf.zeros([784, 10]))
# b = tf.Variable(tf.random_uniform([10], -1., 1.)) //bias 제외 

# y_pred = tf.add(tf.matmul(x ,W), b)
y_pred = tf.matmul(x, W)
y_true = tf.placeholder(tf.float32, [None, 10])

# Cost(Loss) Function
cross_entropy = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred, labels=y_true))

# Gradient Step(Optimze) learning_rate는 하이퍼파라미터이므로 임의 설정할 수 있음
gd_step = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

In [9]:
#Blas GEMM launch failed 발생하여 추가함
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [10]:
with tf.Session() as sess:
    # 학습
    sess.run(tf.global_variables_initializer())
    
    for step in range(NUM_STEPS):
        batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})
        
        if(step + 1) % 100 == 0:
            print(step+1, '|', sess.run(cross_entropy, feed_dict={x: batch_xs, y_true: batch_ys}))
        
    # 테스트
    ans = sess.run(accuracy, feed_dict={x: data.test.images,
                                       y_true: data.test.labels})
    
print("Accuracy: {:.2f}%".format(ans*100))

100 | 0.5691837
200 | 0.52266634
300 | 0.4149245
400 | 0.43970746
500 | 0.3711055
600 | 0.25304908
700 | 0.2846405
800 | 0.24398628
900 | 0.26426652
1000 | 0.27891606
Accuracy: 90.76%


## 2.5 `tf.data.Dataset`을 이용한 MNIST 데이터 불러오기

에러가 발생이 아래의 문구가 나올때(텐서플로 1.8버전)

```
read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
```

따라서, 이러한 경고문구가 나오지 않도록 `tf.keras.datasets.mnist.load_data()` 와 `tf.data.Dataset`을 이용하여 모델의 입력값에 맞게끔 데이터를 만들어 보도록 하자. 아래의 코드들은 tensorflow.org의 [Datasets Quick Start](https://www.tensorflow.org/get_started/datasets_quickstart)를 참고하여 작성했다. 자세한 내용은 해당링크를 참고하면 된다.

In [11]:
def mnist_load():
    (train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

    # Train set
    # Image
    train_x = train_x.reshape([train_x.shape[0], -1])
    train_x = train_x.astype('float32') / 255
    # Label
    train_y = tf.keras.utils.to_categorical(train_y, num_classes=10)

    # Test set
    # Image
    test_x = test_x.reshape([test_x.shape[0], -1])
    test_x = test_x.astype('float32') / 255
    # Label
    test_y = tf.keras.utils.to_categorical(test_y, num_classes=10)
    
    return (train_x, train_y), (test_x, test_y)

In [12]:
(train_x, train_y), (test_x, test_y) = mnist_load()

buffer_size = train_x.shape[0] + test_x.shape[0]

dataset = tf.data.Dataset.from_tensor_slices(({"image": train_x}, train_y))
dataset = dataset.shuffle(buffer_size).repeat().batch(MINIBATCH_SIZE)
iterator = dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

In [14]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))

y_pred = tf.matmul(x ,W)
y_true = tf.placeholder(tf.float32, [None, 10])

# Cost(Loss) Function
cross_entropy = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred, labels=y_true))

# Gradient Step(Optimzer)
gd_step = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

In [17]:
%%time
with tf.Session() as sess:
    # session에 넣어서 학습
    sess.run(tf.global_variables_initializer())
    
    for step in range(NUM_STEPS):
        batch_xs, batch_ys = sess.run(next_batch)
        sess.run(gd_step, feed_dict={x: batch_xs['image'], y_true: batch_ys})
        
        if(step + 1) % 100 == 0:
            print(step+1, '|', sess.run(cross_entropy, feed_dict={x: batch_xs['image'], y_true: batch_ys}))
        
    # 테스트
    ans = sess.run(accuracy, feed_dict={x: test_x,
                                       y_true: test_y})
    
print("Accuracy: {:.2f}%".format(ans*100))

100 | 0.5259843
200 | 0.427052
300 | 0.56716865
400 | 0.5012867
500 | 0.348852
600 | 0.43042976
700 | 0.388591
800 | 0.3226711
900 | 0.30149305
1000 | 0.3112151
Accuracy: 90.80%
Wall time: 18 s
